In [4]:
from selenium.common import JavascriptException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

#driver = webdriver.Chrome('/Users/a2/tools/chromedriver')
driver = webdriver.Safari()
driver.get(f"https://www.immowelt.de")
def button():
    try:
        driver.execute_script('document.querySelector("#usercentrics-root").shadowRoot.querySelector("#uc-center-container > div.sc-ikJyIC.gpszuz > div > div > div > button.sc-gsDKAQ.fILFKg").click()')
    except JavascriptException:


        
        button()
button()
driver.maximize_window()

In [5]:
import json
_time = 1671801363
with open(f'/Users/a2/.cache/immodynamics/{_time}/links.json', "r") as f:
    filed = json.load(f)
for n, i in enumerate(filed):
    del filed[0]
    if n== 4:
        break
len(filed)


25518

In [6]:
del filed[0]
import pandas as pd
import numpy as np
from selenium.common.exceptions import NoSuchElementException
import os
import time
from selenium.webdriver.common.by import By
def get_id(driver, s): 
    if s == "immowelt":
        raw = driver.current_url
        return f"iwe{raw[-7:]}"

# def first_seen(id):
#     try:
#         dirs = os.listdir(f"/Users/a2/.cache/immodynamics/{id}")
#     except FileNotFoundError:
#         return None
#     else:
#         dirs.sort()
#         #catch wrong files
#         for i in dirs:
#             if os.path.isdir(f"/Users/a2/.cache/immodynamics/{id}/{i}"):
#                 return i


def get_en_wrapper(driver):
    return driver.find_element(By.XPATH, '//*[@id="js_innerBody"]/div[2]/main/app-expose/div[3]/div[3]/sd-container[1]/sd-row[6]/sd-col/app-energy/sd-card')
def get_ensource(driver):
    try:
        wrapper = get_en_wrapper(driver)
        h3 = wrapper.find_element(By.TAG_NAME, 'h3')
        div = driver.execute_script("return arguments[0].nextElementSibling", h3)
        et = div.find_elements(By.TAG_NAME, 'p')[1]
        return et.text
    except NoSuchElementException:
        return

def get_enbed(driver):
    try:
        wrapper = get_en_wrapper(driver)
        ps = wrapper.find_elements(By.TAG_NAME, 'p')

        for p in ps:
            if p.text == 'Endenergiebedarf':
                return int(driver.execute_script("return arguments[0].nextElementSibling", p).text)
            if  p.text == 'Endenergieverbrauch':
                return int(driver.execute_script("return arguments[0].nextElementSibling", p).text.split(','))
    except:
        return 0


def get_effclass(driver):
    try:
        wrapper = get_en_wrapper(driver)
        ps = wrapper.find_elements(By.TAG_NAME, 'p')

        for p in ps:
            if p.text == 'Effizienzklasse':
                return driver.execute_script("return arguments[0].nextElementSibling", p).text
    except:
        enbed = get_enbed(driver)
        if enbed != 0:
            if enbed<30:
                return 0
            if enbed<50:
                return 1
            if enbed<75:
                return 2
            if enbed<100:
                return 3
            if enbed<130:
                return 4
            if enbed<160:
                return 5
            if enbed<200:
                return 6
            if enbed<250:
                return 7
            if enbed<300:
                return 8
            else:
                return -1
        else:
            return -1

def get_built(driver):
    wrapper = driver.find_element(By.XPATH, '//*[@id="aImmobilie"]/sd-card')
    spans = wrapper.find_elements(By.TAG_NAME, 'span')
    for s in spans:
        if s.text.startswith('Baujahr:'):
            li = s.find_element(By.XPATH, '..')
    return int(li.text[-4:].split(",")[0].replace('.', ""))



def decode_expose_immowelt(driver):
    time.sleep(2)
    df = pd.DataFrame()
    df["id"] = np.array([get_id(driver, "immowelt")], dtype=str)
    df["url"] = np.array([driver.current_url])
    temp = driver.find_element(By.XPATH, "//*[@id=\"aUebersicht\"]/app-hardfacts/div/div/div[1]/div[1]/strong").get_attribute("innerHTML")[:-2]
    temp = temp.split('&')[0].split(',')[0].replace('.', "")
    df["preis"] = np.array([int(temp)], dtype=np.int16)    
    
    df["typ"] = np.array([1], dtype=np.int8)
    df["qm"] = np.array([int(float((driver.find_element(By.XPATH, "//*[@id=\"aUebersicht\"]/app-hardfacts/div/div/div[2]/div[1]/span").get_attribute("innerHTML")[:-3]).replace(".", "").replace(",", ".")))], dtype=np.int16)
    df["zimmer"] = np.array([int(float(driver.find_element(By.XPATH, "//*[@id=\"aUebersicht\"]/app-hardfacts/div/div/div[2]/div[2]/span").get_attribute("innerHTML").replace(".", "").replace(",", ".")))], dtype=np.int8)
    
    df["plz"] = np.array([int(driver.find_element(By.XPATH, '//*[@id="exposeAddress"]/sd-cell/sd-cell-row/sd-cell-col[2]/span[2]/div[1]').get_attribute("innerHTML")[:5])], dtype=np.int32)

    try:
        driver.find_element(By.XPATH, "//*[@id=\"js_innerBody\"]/div[2]/main/app-expose/div[3]/div[3]/sd-container[1]/sd-row[7]/sd-col/app-details/sd-card/app-texts/sd-read-more[1]/a").click()
    except:
        try:
            time.sleep(0.5)
            driver.find_element(By.XPATH, "//*[@id=\"js_innerBody\"]/div[2]/main/app-expose/div[3]/div[3]/sd-container[1]/sd-row[7]/sd-col/app-details/sd-card/app-texts/sd-read-more[1]/a").click()
        except: 
            pass

    time.sleep(1)

    df["description"] = np.array([driver.find_element(By.XPATH, "//*[@id=\"js_innerBody\"]/div[2]/main/app-expose/div[3]/div[3]/sd-container[1]/sd-row[7]/sd-col/app-details/sd-card/app-texts/sd-read-more[1]/div").text])
    
    df['ensource'] = np.array([get_ensource(driver)], dtype=str)
    df['effclass'] = np.array([get_effclass(driver)], dtype=np.float64)
    df['enbed'] = np.array([get_enbed(driver)], dtype=np.int16)
    df['built'] = np.array([get_built(driver)], dtype=np.int16)

    return df
links = filed
def extract_data(driver):
    return decode_expose_immowelt(driver)

def export_immo(propys, _time):
    import pandas as pd
    df = pd.DataFrame(propys)
    df.to_csv(f"/Users/a2/.cache/immodynamics/{_time}/properties{propys['id']}.csv")
    try:
        return propys["id"]
    except:
        return


for l in links:
    if l.startswith("https://www.immowelt.de/expose/"):
        f = ['https://www.immowelt.de/expose/279my5j', 'https://www.immowelt.de/expose/27ehy5j', 'https://www.immowelt.de/expose/2mfmj4n']
        if l not in f:
            driver.get(l)
    else:
        continue
    try:
        immo_anzeige = extract_data(driver)
    except (AssertionError) as E:
        print(E)
        continue
    id = export_immo(immo_anzeige, _time)

ValueError: could not convert string to float: 'H'

In [12]:
from selenium.common import JavascriptException
from selenium import webdriver
driver2 = webdriver.Chrome('/Users/a2/tools/chromedriver')
driver2.get('https://www.google.com')

/var/folders/nd/5lmcqchd27b_ctzb2whh36800000gn/T/ipykernel_14519/1421785319.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome('/Users/a2/tools/chromedriver')


In [5]:

driver2 = webdriver.Safari()
driver2.get('https://www.google.com')

SessionNotCreatedException: Message: Could not create a session: The Safari instance is already paired with another WebDriver session.
